In [1]:
import os
import re
import ast
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from nltk.tokenize import RegexpTokenizer
from preprocess import preprocess 
import matplotlib.pyplot as plt

train_data = pd.read_csv(os.path.join("data", "train.csv"))
test_data = pd.read_csv(os.path.join("data", "test.csv"))
train_data.dropna(inplace=True)
X, y = train_data.drop(["price"], axis=1), train_data["price"]
y = y.to_numpy().astype(int)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)
X_test = test_data

X_train = preprocess(X_train)

In [2]:
X_train = preprocess(X_train).to_numpy().astype(float)
X_val = preprocess(X_val).to_numpy().astype(float)

# scale
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [5]:
preprocess(X_train)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [3]:
knn = KNeighborsClassifier()

n_neighbors = list(range(1,10))
hyperparameters = dict(n_neighbors=n_neighbors)
clf = GridSearchCV(knn, hyperparameters, cv=5)

best_knn = clf.fit(X_train,y_train)

print('Best K neighbors:', best_knn.best_estimator_.get_params()['n_neighbors'])

Best K neighbors: 9


In [4]:
y_pred = best_knn.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.78      0.71       804
           1       0.34      0.38      0.36       557
           2       0.31      0.26      0.28       514
           3       0.37      0.36      0.37       514
           4       0.35      0.27      0.31       314
           5       0.71      0.47      0.57       232

    accuracy                           0.46      2935
   macro avg       0.45      0.42      0.43      2935
weighted avg       0.45      0.46      0.45      2935



In [11]:
X_test = pd.read_csv(os.path.join("data", "test.csv"))
X_test = scaler.transform(preprocess(X_test))
test_predictions = best_knn.predict(X_test)

submission = {
    "id" : list(range(len(test_predictions))), 
    "price" : list(test_predictions.astype(float))
}

submission = pd.DataFrame.from_dict(submission)

submission.to_csv("bahng_2.csv", index=False)